In [ ]:
import os

def count_files_in_directory(root_folder):
    total_files = 0

    for dirpath, dirnames, filenames in os.walk(root_folder):
        total_files += len(filenames)

    return total_files

root_folder_path = '/content/drive/MyDrive/SCVD/SCVD_converted/TRAIN3'
total_files = count_files_in_directory(root_folder_path)

print(f'Total number of files in the directory: {total_files}')


Total number of files in the directory: 124


In [ ]:
import os
import numpy as np
import cv2
from skimage.transform import resize
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
fights_train = np.zeros((124, 30, 160, 160, 3), dtype=np.float)
labels_train = []

<ipython-input-5-c2ba5d875c97>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fights_train = np.zeros((124, 30, 160, 160, 3), dtype=np.float)


In [ ]:
def capture(filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
      #  plt.imshow(frame)
      #  plt.show()
    # print(filename)
    frm = resize(frame,(160, 160, 3))
    frm = np.expand_dims(frm,axis=0)
    # plt.imshow(frame)
    # plt.show()
    if(np.max(frm)>1):
        frm = frm/255.0
    # print(frm)
    frames[i][:] = frm
    i +=1
    while i < 30:
        rval, frame = vc.read()
          # plt.imshow(frame)
          # plt.show()
        # print(filename)
        #print(i)
        # plt.imshow(frame)
        # plt.show()
        #frm = cv2.resize(frame,(200,200))
        frm = resize(frame,(160, 160, 3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        # print(frm)
        frames[i][:] = frm
        i +=1
    vc.release()
    return frames

def cut_save(main_dir,mod):
    i = 0
    dem1 = 0
    #fights = np.zeros((399, 40, 200, 200, 3), dtype=np.float)
    #noFights = np.zeros((599, 42, 200, 200, 3), dtype=np.float)
    for x in os.listdir(main_dir):
        if 1 == 1:
            td = main_dir+'/'+x
            #for y in os.listdir(main_dir+x+'/'):
                #print(y)
            for file in os.listdir(td):
                fl = os.path.join(td, file)
                videos = capture(fl)
                if mod == 'Train':
                    fights_train[i][:][:] = videos
                    i +=1
                    if x =='Normal':
                        labels_train.append(0)
                        dem1+=1
                        print(dem1)
                    else:
                        labels_train.append(1)
                        dem1+=1
                        print(dem1)
                elif mod =='Test':
                    fights_test[i][:][:] = videos
                    i +=1
                    if x =='Normal':
                        labels_test.append(0)
                    else:
                        labels_test.append(1)
                # elif mod =='val':
                #     fights_val[i][:][:] = videos
                #     i +=1
                #     if x =='fights':
                #         labels_val.append(1)
                #     else:
                #         labels_val.append(0)

In [ ]:
cut_save('/content/drive/MyDrive/SCVD/SCVD_converted/TRAIN3',"Train")

<ipython-input-6-1edadafdb523>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frames = np.zeros((30, 160, 160, 3), dtype=np.float)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [ ]:
X_train=fights_train
y_train=labels_train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, fights_test, y_train, labels_test = train_test_split(fights_train,labels_train, test_size=0.33, random_state=42)

In [ ]:
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks
layers = tf.keras.layers
models = tf.keras.models
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
losses = tf.keras.losses
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks

plot_model = tf.keras.utils.plot_model

In [ ]:
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
num_classes = 2
cnn = models.Sequential()
    #cnn.add(base_model)

input_shapes=(160,160,3)
np.random.seed(1234)
vg19 = tf.keras.applications.vgg19.VGG19
base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))
    # Freeze the layers except the last 4 layers
for layer in base_model.layers:
  layer.trainable = False

cnn = models.Sequential()
cnn.add(base_model)
cnn.add(layers.Flatten())
model = models.Sequential()

model.add(layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
model.add(layers.LSTM(30 , return_sequences= True))

model.add(layers.TimeDistributed(layers.Dense(90)))
model.add(layers.Dropout(0.1))

model.add(layers.GlobalAveragePooling1D())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(num_classes, activation="sigmoid"))

adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.load_weights('/content/drive/MyDrive/SCVD/Model/model-cctv-train2.keras')
rms = optimizers.RMSprop()

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
model.summary()

80134624/80134624 [==============================] - 0s 0us/step


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 30, 12800)         20024384  
 ributed)                                                        
                                                                 
 lstm (LSTM)                 (None, 30, 30)            1539720   
                                                                 
 time_distributed_1 (TimeDi  (None, 30, 90)            2790      
 stributed)                                                      
                                                                 
 dropout (Dropout)           (None, 30, 90)            0         
                                                                 
 global_average_pooling1d (  (None, 90)                0         
 GlobalAveragePooling1D)                                         
                                                      

In [ ]:
callbacks = tf.keras.callbacks
utils = tf.keras.utils
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=8,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('model1.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_loss',patience=1, verbose=2,factor=0.5,min_lr=0.0000001)


In [ ]:
batch_size = 5
epochs = 20

In [ ]:
y_train = utils.to_categorical(labels_train)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [ ]:
utils = tf.keras.utils
y_test = utils.to_categorical(labels_test)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [ ]:
model.fit(fights_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(fights_test, y_test),callbacks=[earlyStopping,mcp_save, reduce_lr_loss,history]) #mcp_save,

Epoch 1/20
25/25 [==============================] - 51s 1s/step - loss: 1.4712 - accuracy: 0.4839 - val_loss: 0.7391 - val_accuracy: 0.5610 - lr: 0.0010
Epoch 2/20
25/25 [==============================] - 15s 599ms/step - loss: 0.7406 - accuracy: 0.5000 - val_loss: 0.6776 - val_accuracy: 0.5854 - lr: 0.0010
Epoch 3/20
25/25 [==============================] - 15s 618ms/step - loss: 0.6722 - accuracy: 0.5323 - val_loss: 0.6543 - val_accuracy: 0.6341 - lr: 0.0010
Epoch 4/20
25/25 [==============================] - 14s 548ms/step - loss: 0.6456 - accuracy: 0.5968 - val_loss: 0.6132 - val_accuracy: 0.6585 - lr: 0.0010
Epoch 5/20
25/25 [==============================] - 15s 623ms/step - loss: 0.6159 - accuracy: 0.6452 - val_loss: 0.4741 - val_accuracy: 0.7805 - lr: 0.0010
Epoch 6/20
25/25 [==============================] - 14s 558ms/step - loss: 0.3742 - accuracy: 0.8065 - val_loss: 0.3205 - val_accuracy: 0.8293 - lr: 0.0010
Epoch 7/20
25/25 [==============================] - 16s 639ms/step 

In [ ]:
fights_test = []

In [ ]:
acc = history.acc
val_acc = history.val_acc
loss = history.loss
val_loss = history.val_loss
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# history = model.fit_generator(aug.flow(X_train, y_train, batch_size=64), epochs=50, validation_data=aug.flow(X_test,y_test, batch_size=128))

# def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
#     fig, axs = plt.subplots(1, 2, figsize=(15, 5))
#     axs[0].plot(range(1, len(model_history.history[acc]) + 1), model_history.history[acc])
#     axs[0].plot(range(1, len(model_history.history[val_acc]) + 1), model_history.history[val_acc])
#     axs[0].set_title('Model Accuracy')
#     axs[0].set_ylabel('Accuracy')
#     axs[0].set_xlabel('Epoch')
#     axs[0].set_xticks(np.arange(1, len(model_history.history[acc]) + 1), len(model_history.history[acc]) / 10)
#     axs[0].legend(['train', 'val'], loc='best')
#     axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
#     axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
#     axs[1].set_title('Model Loss')
#     axs[1].set_ylabel('Loss')
#     axs[1].set_xlabel('Epoch')
#     axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']) / 10)
#     axs[1].legend(['train', 'val'], loc='best')
#     plt.show()
#     plt.savefig('roc.png')

# plot_model_history(history)

In [ ]:
fights_train = []

In [ ]:
score = model.evaluate(fights_test, y_test, batch_size=3)
score

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
Y_pred = model.predict(fights_test , batch_size=1)

In [ ]:
yprd = Y_pred > 0.9
yprd

In [ ]:
ypredicted = []
for zero,one in yprd:
    if zero == True:
        ypredicted.append(0)
    else:
        ypredicted.append(1)

In [ ]:
# ypredicted

In [ ]:
# y_test

In [ ]:
y = []

for zero,one in y_test:
    if zero == True:
        y.append(0)
    else:
        y.append(1)

In [ ]:
confusion = confusion_matrix(y,ypredicted)
confusion.shape

In [ ]:
print_confusion_matrix(confusion, [0,1], figsize = (30,15), fontsize=16)

In [ ]:
print('Classification Report')
print(classification_report(y, ypredicted, target_names=['no-violance','violance']))

In [ ]:
model.save_weights("model.keras")